In [1]:
import pymc as pm
import numpy as np
import arviz as az

%load_ext lab_black
%load_ext watermark

# Priors as hidden mixtures

This example demonstrates ...

Adapted from [unit 7: tasmixture.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit7/tasmixture.odc)


## Associated lecture video: Unit 7 Lesson 4

In [2]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed?v=xomK4tcePmc&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=66" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

Not really a problem statement - this is just to demonstrate the equivalence of a couple methods. Probably add notes and explanation here. Variable names from original problem are retained right now, but they are a mess. Fix later.

In [3]:
df = 6
x = y = 10
mu0 = 6
tau1 = 10
tau = 0.4
a = df / 2
b = df / (2 * tau)

with pm.Model() as m:
    mu1 = pm.StudentT("mu1", nu=df, mu=mu0, sigma=1 / (tau**0.5))
    X = pm.Normal("X", mu1, tau1, observed=x)

    prec = pm.Gamma("prec", a, b)
    mu2 = pm.Normal("mu2", mu0, tau=prec)
    Y = pm.Normal("Y", mu2, tau1, observed=y)

    trace = pm.sample(3000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/Users/aaron/mambaforge/envs/pymc/lib/python3.10/site-packages/pymc/model.py:984: FutureWarning: `Model.initial_point` has been deprecated. Use `Model.recompute_initial_point(seed=None)`.
  warnings.warn(
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu1, prec, mu2]
/Users/aaron/mambaforge/envs/pymc/lib/python3.10/site-packages/pymc/model.py:984: FutureWarning: `Model.initial_point` has been deprecated. Use `Model.recompute_initial_point(seed=None)`.
  warnings.warn(


/Users/aaron/mambaforge/envs/pymc/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 11 seconds.


In [4]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu1,6.156,1.884,2.753,9.966,0.021,0.015,9287.0,6550.0,1.0
prec,0.408,0.238,0.054,0.844,0.003,0.002,6522.0,6125.0,1.0
mu2,6.109,1.865,2.549,9.551,0.023,0.018,7623.0,5322.0,1.0


Results are not equivalent to BUGS...why?